### Driver Code

In [1]:
import pandas as pd, os, inspect
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from main import opener, escalator
os.environ['OPENAI_API_KEY'] = "insert your key"

#### Opener and Escalator parameters

In [9]:
opener_params = inspect.signature(opener)
print("Attribute Name and Default Value:")
for i in opener_params.parameters:
    print(i + " = " + str(opener_params.parameters[i].default) + " " + str(type(escalator_params.parameters[i].default)))

Attribute Name and Default Value:
prompt_path = opener_prompt.md <class 'str'>
leads_path = leads.xlsx <class 'str'>
opener_path = opener_output.xlsx <class 'str'>
model_name = gpt-3.5-turbo <class 'str'>
model_temperature = 0 <class 'float'>
model_max_tokens = 300 <class 'int'>


In [8]:
escalator_params = inspect.signature(escalator)
print("Attribute Name and Default Value:")
for i in escalator_params.parameters:
    print(i + " = " + str(escalator_params.parameters[i].default) + " " + str(type(escalator_params.parameters[i].default)))

Attribute Name and Default Value:
prompt_path = escalator_prompt.md <class 'str'>
leads_path = leads.xlsx <class 'str'>
opener_path = opener_output.xlsx <class 'str'>
escalator_path = escalator_output.xlsx <class 'str'>
model_name = gpt-3.5-turbo <class 'str'>
model_temperature = 0.2 <class 'float'>
model_max_tokens = 300 <class 'int'>


#### Generation

In [4]:
opener()

Loading template and leads...
Opener Template loaded!
Opener Leads loaded!
Making template...
Opener Prompts generated!
Opener Generating emails...
Opener Emails generated!
Opener output saved to:  opener_output.xlsx


In [5]:
escalator()

Escalator Loading template and leads...
Escalator Template loaded!
Escalator Leads loaded!
Making template...
Escalator Prompts generated!
Escalator Emails generated!
